# Import libraries

In [37]:
from aquacrop import AquaCropModel, Soil, Crop, InitialWaterContent, GroundWater, IrrigationManagement
from aquacrop.utils import prepare_weather, get_filepath
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
import pathlib
from pathlib import Path
import geopandas as gpd

import xarray as xr

import rasterio
from rasterio.plot import show

from matplotlib import pyplot

import numpy as np

import glob as glob

import rioxarray

from pathlib import Path

import zipfile
import rioxarray as rio 
from tqdm import tqdm
from datetime import datetime as dt
import datetime
import netCDF4 as nc
from netCDF4 import Dataset
import os, sys
import copy
import platform
import tempfile
import logging
from math import log10, cos, sin, asin, sqrt, exp, pi, radians
from bisect import bisect_left
import textwrap

import traceback

from datetime import date, timedelta

import json

from tqdm import tqdm

# Directories

In [38]:
HomeDir = '/home/c4ubuntu/c4gdata/SADC_Agri/'

In [39]:
DataDir = '/home/c4ubuntu/c4gdata/SADC_Agri/Data/'

# Functions

In [40]:
def get_soil_name(lon, lat):
    Soils = {1:'SiltClay',2:'SandyClay',3:'ClayLoam',4:'SiltClayLoam',5:'SandyClayLoam',6:'Loam',7:'SandyLoam',8:'LoamySand',9:'Sand'}
    #Soil Class
    pnt_soil = gyga_soilclass.sel(x=lon,y=lat,  method="nearest")
    try:    
        pnt_class = int(pnt_soil.band_data.values)
        pnt_class_name = Soils.get(pnt_class)
    except:
        pnt_class_name = pnt_soil.band_data.values
    return(pnt_class_name)

In [41]:
def process_one_path(path):
    # use a context manager, to ensure the file gets closed after use
    with xr.open_dataset(path) as ds:
        ds.load()
        return ds

In [42]:
gyga_soilclass = xr.open_dataset(os.path.join(DataDir,'GIS/Soil/gyga_30cm_texclss_1km.tif'))
    
def get_soil_profile(lon, lat):
    Soils = {1:'SiltClay',2:'SandyClay',3:'ClayLoam',4:'SiltClayLoam',5:'SandyClayLoam',6:'Loam',7:'SandyLoam',8:'LoamySand',9:'Sand'}
    #Soil Class
    pnt_soil = gyga_soilclass.sel(x=lon,y=lat,  method="nearest")
    pnt_class = int(pnt_soil.band_data.values)
    pnt_class_name = Soils.get(pnt_class)
    # Custom Soil
    point_soil = Soil(soil_type=pnt_class_name, dz=[0.05]*2+[0.15]*2+[0.30]*2+[0.6]*2+[1.0]*2+[2.0]*2)
    
    depths = {'1':'0.5','2':'0.15','3':'0.30','4':'0.60','5':'1','6':'2'}
    
    for level in range(1,7):
        for var in ['SND','CLY']:
            filename = 'af_{}PPT_T__M_sd{}_250m_reprojected.tif'.format(var,level)
            if var == 'SND':
                sandtemp = process_one_path(os.path.join(DataDir,'GIS/Soil',filename))
                sandtemp = sandtemp.rio.write_crs(countries_adm.crs)
                sandtempval = sandtemp.sel(x=lon,y=lat,  method="nearest")
                sandtempdata = int(sandtempval.band_data.values)
               # sandtempdata = str(sandtempdata)
               # print(var, 'Level:',level, 'Value: ',sandtempdata)
            elif var == 'CLY':
                claytemp = process_one_path(os.path.join(DataDir,'GIS/Soil',filename))
                claytemp = claytemp.rio.write_crs(countries_adm.crs)
                claytempval = claytemp.sel(x=lon,y=lat,  method="nearest")
                claytempdata = int(claytempval.band_data.values)
               # claytempdata = str(claytempdata)
               # print(var, 'Level:',level, 'Value: ',claytempdata)
        dep = depths.get('{}'.format(level))
        dep = float(dep)
        point_soil.add_layer_from_texture(thickness=dep, Sand=sandtempdata, Clay=claytempdata,OrgMat=2.5,penetrability=100)
        
        return(point_soil)

In [43]:
def get_plantdate_month(lat, lon, crop, irr_stat, weatherfile):
    calendar = xr.open_dataset(os.path.join(DataDir,'Crop/Crop_Calendar/{}_{}_ggcmi_crop_calendar_phase3_v1.01.nc4').format(crop, irr_stat))
    pnt_date = calendar.sel(lon=lon,lat=lat,  method="nearest")
    pnt_date = pnt_date.planting_day.values
    sdate = weatherfile.Date.dt.year.unique()[0]
    strt_date = date(int(sdate), 1, 1)
    plant_date = strt_date + timedelta(days=int(pnt_date) - 1)
    plant_date = plant_date.strftime("%m-%d")
    return(plant_date)

In [44]:
def get_plantcalendardate(lat, lon, crop, irr_stat, weatherfile):
    calendar = xr.open_dataset(os.path.join(DataDir,'Crop/Crop_Calendar/{}_{}_ggcmi_crop_calendar_phase3_v1.01.nc4').format(crop, irr_stat))
    pnt_date = calendar.sel(lon=lon,lat=lat,  method="nearest")
    pnt_date = pnt_date.planting_day.values
    sdate = weatherfile.Date.dt.year.unique()[0]
    strt_date = date(int(sdate), 1, 1)
    plant_date = strt_date + timedelta(days=int(pnt_date) - 1)
    plant_date = plant_date.strftime("%d-%m-%Y")
    return(plant_date)

In [45]:
def aggregate_yields(output_dir):
    out_files = glob.glob(os.path.join(DataDir,output_dir,'*.csv'))
    country_yield = pd.DataFrame()
    for file in out_files:
        dftmp = pd.read_csv(file, index_col=3)
        dftmp = dftmp[['crop Type',
           'Yield (tonne/ha)', 'Lat', 'Lon',
           'Watershed_No', 'Scenario']]
        if len(country_yield) == 0:
            country_yield = dftmp
        else:
            country_yield = pd.concat([country_yield, dftmp])
            
    country_yield.reset_index(inplace=True)
    country_yield.Lat = country_yield.Lat.astype(float)
    country_yield.Lon = country_yield.Lon.astype(float)
    country_yield.set_index(['Harvest Date (YYYY/MM/DD)','Lat','Lon','Watershed_No', 'Scenario','crop Type'], inplace=True)
    ds = country_yield.to_xarray()
    ds = ds.rename({'Harvest Date (YYYY/MM/DD)':'Year','Yield (tonne/ha)':'Yield (t/ha)'})
    return(ds)

In [46]:
def get_plantingdates(lat, lon, weatherfile, cropname):
    planting_dates = {}
    for year in weatherfile.Date.dt.year.unique():
        if year != 2022:
            year_clim = weatherfile.set_index('Date').loc['{}'.format(year)]
            year_clim.reset_index(inplace=True)
            plant_date = get_plantcalendardate(lat=lat, lon=lon, crop=cropname, irr_stat=irr_status, weatherfile=year_clim)
            start_window = pd.to_datetime(plant_date, infer_datetime_format=True)-pd.DateOffset(90)
            end_window = pd.to_datetime(plant_date, infer_datetime_format=True)+pd.DateOffset(90)
            window_clim_01 = year_clim.set_index('Date').resample('7D').sum()
            window_clim_02 = year_clim.set_index('Date').resample('14D').sum()
            arex_01 = window_clim_01[start_window:end_window].loc[window_clim_01['Precipitation']>25]
            arex_02 = window_clim_02[start_window:end_window].loc[window_clim_02['Precipitation']>40]
            arex = [i for i in arex_01.index if i in arex_02.index]
            if len(arex) > 0:
                arex = arex[0]+pd.DateOffset(7)
            else:
                try:
                    arex = arex_01.index[0]
                except:
                    arex = pd.to_datetime(plant_date, infer_datetime_format=True)
            planting_dates[year] = arex.strftime("%m-%d")
    return(planting_dates)

In [47]:
def run_aquacrop(country, scenario, wshed, start_sim, end_sim, weatherfile, soil, crop, initial_water, irrigation):
    model = AquaCropModel(sim_start_time=f'{start_sim}',
                                  sim_end_time=f'{end_sim}',
                                  weather_df=weatherfile,
                                  soil=soil,
                                  crop=crop,
                                  initial_water_content=initial_water,
                                  irrigation_management=irrigation)               
    try:
        model.run_model(till_termination=True)
        print('Model ran for ', lat, lon, year)
        out = model._outputs.final_stats
        out['Lat'] = lat
        out['Lon'] = lon
        out['Watershed_No'] = wshed
        scene = str(scenario.get('Name'))
        out['Scenario'] = scene
        out['Country'] = country
        out['Year'] = out['Harvest Date (YYYY/MM/DD)'].dt.year
    except:
        traceback_output = traceback.format_exc()
        print('Model didnt run for:', lat,lon,'Error:', traceback_output)
    return(out)

In [48]:
def get_startdate(year):
    try:
        start_sim = str(year) + '-' + planting_dates.get(year)
        start_sim = pd.to_datetime(start_sim)
        start_sim = start_sim - pd.DateOffset(90)
        start_sim = start_sim.strftime('%Y/%m/%d')
    except:
        print('No planting date: ', year, planting_dates.get(year))
    return(start_sim)

In [49]:
def get_enddate(year):
    try:
        end_sim = str(year) + '-' + planting_dates.get(year)
        end_sim = pd.to_datetime(end_sim)
        end_sim = end_sim + pd.DateOffset(365)
        end_sim = end_sim.strftime('%Y/%m/%d')
    except:
        print('No planting date: ', year, planting_dates.get(year))
    return(end_sim)

# Create Gridpoints

In [50]:
os.listdir(Path(DataDir,'Climate/Sitefiles/CMIP6/Zambia'))

['ACCESS-CM2',
 'CMCC-ESM2',
 'INM-CM4-8',
 'INM-CM5-0',
 'KACE-1-0-G',
 'KIOST-ESM',
 'MIROC-ES2L',
 'MIROC6',
 'MPI-ESM1-2-LR',
 'MRI-ESM2-0',
 'NorESM2-MM',
 'UKESM1-0-LL']

In [51]:
country = 'Zambia'

In [21]:
# Models for timesteps and countries

eswatini_245 = ['CMCC-ESM2',
             'INM-CM5-0',
             'KACE-1-0-G',
            'MRI-ESM2-0',
           'NorESM2-MM']

eswatini_585 = ['UKESM1-0-LL',
                'CMCC-ESM2',
                'KACE-1-0-G',
                'MPI-ESM1-2-LR',
                'INM-CM5-0']


kafue_245 = ['ACCESS-CM2',
             'INM-CM5-0',
             'KACE-1-0-G',
             'MIROC-ES2L',
             'MRI-ESM2-0']

kafue_585 =['MRI-ESM2-0',
           'KACE-1-0-G',
           'NorESM2-MM',
            'MIROC-ES2L',
           'INM-CM5-0']

lesotho_245 = ['CMCC-ESM2',
               'INM-CM5-0',
               'KACE-1-0-G',
               'MRI-ESM2-0',
               'NorESM2-MM']

lesotho_585 = ['NorESM2-MM',
              'KACE-1-0-G',
              'MRI-ESM2-0',
              'INM-CM5-0']

luangwa_245 = ['CMCC-ESM2',
               'INM-CM5-0',
               'KACE-1-0-G',
               'MRI-ESM2-0',
               'UKESM1-0-LL']

luangwa_585 = ['MRI-ESM2-0',
               'UKESM1-0-LL',
               'MPI-ESM1-2-LR',
               'INM-CM5-0',
               'MIROC-ES2L']

malawi_245 = ['CMCC-ESM2',
               'INM-CM5-0',
               'KACE-1-0-G',
               'MRI-ESM2-0',
               'UKESM1-0-LL']

malawi_585 = ['MRI-ESM2-0',
              'UKESM1-0-LL',
              'MIROC-ES2L',
              'MPI-ESM1-2-LR',
              'INM-CM4-8']

madagascar_245 = ['MPI-ESM1-2-LR',
                  'UKESM1-0-LL',
                  'CMCC-ESM2',
                  'ACCESS-CM2',
                  'INM-CM4-8']

madagascar_585 = ['UKESM1-0-LL',
                  'KACE-1-0-G',
                  'CNRM-ESM2-1',
                  'MRI-ESM2-0',
                  'INM-CM4-8']

In [22]:
models = luangwa_245

In [23]:
countries_points_dict = []
for model in models[:1]:
    model_files = os.listdir(Path(DataDir,'Climate/Sitefiles/CMIP6/Zambia', model))
    for line in model_files:
        latlon = {}
        lat = line.split('_')[-2]
        lon = line.split('_')[-1].replace('.csv', '')
        latlon['lat'] = float(lat)
        latlon['lon'] = float(lon)
        latlon['ADMIN'] = country
        countries_points_dict.append(latlon)

In [24]:
countries_points_dict

[{'lat': -10.0, 'lon': 32.8, 'ADMIN': 'Zambia'},
 {'lat': -10.0, 'lon': 32.9, 'ADMIN': 'Zambia'},
 {'lat': -10.0, 'lon': 33.0, 'ADMIN': 'Zambia'},
 {'lat': -10.0, 'lon': 33.1, 'ADMIN': 'Zambia'},
 {'lat': -10.0, 'lon': 33.2, 'ADMIN': 'Zambia'},
 {'lat': -10.0, 'lon': 33.3, 'ADMIN': 'Zambia'},
 {'lat': -10.1, 'lon': 32.8, 'ADMIN': 'Zambia'},
 {'lat': -10.1, 'lon': 32.9, 'ADMIN': 'Zambia'},
 {'lat': -10.1, 'lon': 33.0, 'ADMIN': 'Zambia'},
 {'lat': -10.1, 'lon': 33.1, 'ADMIN': 'Zambia'},
 {'lat': -10.1, 'lon': 33.2, 'ADMIN': 'Zambia'},
 {'lat': -10.1, 'lon': 33.3, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 32.6, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 32.7, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 32.8, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 32.9, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 33.0, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 33.1, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 33.2, 'ADMIN': 'Zambia'},
 {'lat': -10.2, 'lon': 33.3, 'ADMIN': 'Zambia'},
 {'lat': -10.3, 'lon

## Get the soil name for each point

In [25]:
# Soil Class 
gyga_soilclass = xr.open_dataset(os.path.join(DataDir,'GIS/Soil/gyga_30cm_texclss_1km.tif'))

In [26]:
class NumpyEncoder(json.JSONEncoder):
    """ Special json encoder for numpy types """
    def default(self, obj):
        if isinstance(obj, (np.int_, np.intc, np.intp, np.int8,
                            np.int16, np.int32, np.int64, np.uint8,
                            np.uint16, np.uint32, np.uint64)):
            return int(obj)
        elif isinstance(obj, (np.float_, np.float16, np.float32,
                              np.float64)):
            return float(obj)
        elif isinstance(obj, (np.ndarray,)):
            return obj.tolist()
        return json.JSONEncoder.default(self, obj)

In [27]:
for row in tqdm(countries_points_dict):
    lon = "{:.2f}".format(float(row.get('lon')))
    lat = "{:.2f}".format(float(row.get('lat')))
    # Soil Name
    try:
        point_soil = get_soil_name(lon=lon, lat=lat)
        row['SoilName'] = point_soil
    except:
        row['SoilName'] = 0
        print('Soil name didnt work for', lat,lon)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1862/1862 [00:01<00:00, 1066.93it/s]


## Root zone depth for each point

In [28]:
# Root Zone Depth
gyga_rootzonedepth = xr.open_dataset(os.path.join(DataDir,'GIS/Soil/gyga_af_erzd__m_1km_rootzonedepth.tif'))

In [29]:
DataDir

'/home/c4ubuntu/c4gdata/SADC_Agri/Data/'

In [30]:
for row in tqdm(countries_points_dict):
    lon = "{:.2f}".format(float(row.get('lon')))
    lat = "{:.2f}".format(float(row.get('lat')))
    # Soil Name
    try:
        pnt_rootzoneD = gyga_rootzonedepth.sel(x=lon,y=lat,  method="nearest")
        pnt_rootzoneD = int(pnt_rootzoneD.band_data.values) / 100
        row['RZD'] = pnt_rootzoneD
    except Exception as e:
        row['RZD'] = np.nan
        print(e)
        
json_str = json.dumps(countries_points_dict, cls=NumpyEncoder)

# open file for writing, "w" 
f = open(f"{DataDir}GIS/Gridpoints/{country}_points_dict.json","w")

# write json object to file
f.write(json_str)

# close file
f.close()

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1862/1862 [00:01<00:00, 1134.89it/s]


## Get the soil profile for each point

In [31]:
import json 
with open(f'{DataDir}GIS/Gridpoints/{country}_points_dict.json') as json_file:
    countries_points_dict = json.load(json_file)

len(countries_points_dict)

1862

In [32]:
Combined = xr.Dataset()

for var in ['SND']:
    for level in range(1,3):
        filename = 'af_{}PPT_T__M_sd{}_250m_reprojected.tif'.format(var,level)
        temp = process_one_path(os.path.join(DataDir,'GIS/Soil',filename))
        temp = temp.assign_coords({'Level': level})
        temp = temp.assign_coords({'Type':var})
        
        if len(Combined) == 0:
            Combined = temp
        else:
            Combined = xr.merge([Combined, temp], compat='override')

In [33]:
for row in countries_points_dict:
    row['SND'] = []
    row['CLY'] = []

In [34]:
Combined = xr.Dataset()

for var in tqdm(['SND', 'CLY']):
    for level in range(1,7):
        filename = 'af_{}PPT_T__M_sd{}_250m_reprojected.tif'.format(var,level)
        temp = process_one_path(os.path.join(DataDir,'GIS/Soil',filename))
        temp = temp.assign_coords({'Level': level})
        temp = temp.assign_coords({'Type':var})
        for row in countries_points_dict:
            lon = "{:.2f}".format(float(row.get('lon')))
            lat = "{:.2f}".format(float(row.get('lat')))
            rowsoil = temp.sel(x=lon,y=lat,  method="nearest")
            try:
                rowsoil = float(rowsoil.band_data.values)
            except:
                rowsoil = np.nan
            row[var].append(rowsoil)

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [03:16<00:00, 98.04s/it]


In [35]:
json_str = json.dumps(countries_points_dict, cls=NumpyEncoder)

# open file for writing, "w" 
f = open(f"{DataDir}GIS/Gridpoints/{country}_points_soils_dict.json","w")

# write json object to file
f.write(json_str)

# close file
f.close()

# Planting dates

In [36]:
##This function checks for the first date on which more than 25mm of rainfall is received in 7 days, followed by 40 mm in 14 days
def get_plantingdates(weatherfile):
    planting_dates = {}
    for year in weatherfile.Date.dt.year.unique():
        if year != 2022:
            start_window = f'{year}-08-01'
            end_window = f'{year+1}-04-01'
            year_clim = weatherfile.set_index('Date').loc[start_window:end_window]
            year_clim.reset_index(inplace=True)
            window_clim_01 = year_clim.set_index('Date').resample('7D').sum()
            window_clim_02 = year_clim.set_index('Date').resample('14D').sum()
            arex_01 = window_clim_01[start_window:end_window].loc[window_clim_01['Precipitation']>25]
            arex_02 = window_clim_02[start_window:end_window].loc[window_clim_02['Precipitation']>40]
            arex = [i for i in arex_01.index if i in arex_02.index]
            if len(arex) > 0:
                arex = arex[0]
            else:
                try:
                    arex = arex_01.index[0]
                except:
                    arex = arex_02.index[0]
            planting_dates[str(year)] = arex.strftime("%Y/%m/%d")
    return(planting_dates)

In [5]:
country = 'eSwatini'

models = ['CMCC-ESM2',
         'INM-CM5-0',
         'KACE-1-0-G',
         'MRI-ESM2-0',
         'NorESM2-MM']

planting_dates = []
for model in models[:1]:
    model_files = os.listdir(Path(DataDir,'Climate/Sitefiles/CMIP6/eSwatini', model))
    for line in model_files:
        latlon = {}
        lat = line.split('_')[-2]
        lon = line.split('_')[-1].replace('.csv', '')
        latlon['lat'] = float(lat)
        latlon['lon'] = float(lon)
        latlon['ADMIN'] = country
        planting_dates.append(latlon)

In [33]:
# Climate
data_type = 'CMIP6'

country = 'eSwatini'

models = ['CMCC-ESM2',
         'INM-CM5-0',
         'KACE-1-0-G',
         'MRI-ESM2-0',
         'NorESM2-MM']

for model in models[:1]:
    print(model)
    model_name = model
    for row in tqdm(planting_dates):
        lon = "{:.1f}".format(float(row.get('lon')))
        lat = "{:.1f}".format(float(row.get('lat')))
        clim_filename = os.path.join(DataDir,f'Climate/SiteFiles/{data_type}/{country}/{model_name}',
                                         f'{model_name}_{country}_{lat}_{lon}.csv')
        SiteFile = pd.read_csv(clim_filename,parse_dates=['Date'])
        SiteFile = SiteFile.loc[(SiteFile['Date']>='1990')&(SiteFile['Date']<='2022')]
        SiteFile.drop(SiteFile.columns[[0]], axis=1, inplace=True)
        try:
            row[model] = get_plantingdates(SiteFile)
        except Exception as e:
            print(e)

CMCC-ESM2


 12%|████████████                                                                                    | 10/80 [00:03<00:22,  3.04it/s]

index 0 is out of bounds for axis 0 with size 0


 25%|████████████████████████                                                                        | 20/80 [00:06<00:18,  3.17it/s]

index 0 is out of bounds for axis 0 with size 0


...

 74%|██████████████████████████████████████████████████████████████████████▊                         | 59/80 [00:19<00:06,  3.01it/s]

index 0 is out of bounds for axis 0 with size 0


 75%|████████████████████████████████████████████████████████████████████████                        | 60/80 [00:19<00:06,  3.01it/s]

index 0 is out of bounds for axis 0 with size 0


100%|████████████████████████████████████████████████████████████████████████████████████████████████| 80/80 [00:26<00:00,  3.05it/s]
